# Building Fast Queries on a CSV

Dataset: https://www.kaggle.com/ionaskel/laptop-prices

Background:

* Take the perspective as the owner of an online laptop store
* Want to build a way to answer business questions about inventory
Objectives:

* Create a class that represents our inventory.
* Methods in that class will implement the queries to answer questions about inventory
* Pre-process the inventory data to make queries run faster
Queries to Answer:

* Given a laptop ID, find the corresponding data
* Given an amount of money, find whether there are two laptops whose total price is that given amount
* Identify all laptops whose price falls within a given budget

## Reading the Inventory
1. Use the csv module to read the 'laptops.csv' file
2. Separate the header row
3. Separate the remaining rows

In [1]:
import csv                          # Opened "laptops.csv" and converted it to list of lists

with open('laptops.csv') as laps:
    read = csv.reader(laps)
    file = list(read)
    header = file[0]
    rows = file[1:]
    
print(header)
print(" ")

rows[:2]
    

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
 


[['6571244',
  'Apple',
  'MacBook Pro',
  'Ultrabook',
  '13.3',
  'IPS Panel Retina Display 2560x1600',
  'Intel Core i5 2.3GHz',
  '8GB',
  '128GB SSD',
  'Intel Iris Plus Graphics 640',
  'macOS',
  '1.37kg',
  '1339'],
 ['7287764',
  'Apple',
  'Macbook Air',
  'Ultrabook',
  '13.3',
  '1440x900',
  'Intel Core i5 1.8GHz',
  '8GB',
  '128GB Flash Storage',
  'Intel HD Graphics 6000',
  'macOS',
  '1.34kg',
  '898']]

## Created a Inventory Class

In [2]:
# 1st version of Inventory class
# Opens the csv file and reads the rows

class Inventory():
    
   def __init__(self, csv_filename):
        
       with open(csv_filename) as f:
           reader = csv.reader(f)
           file = list(reader)
       self.header = file[0]
       self.rows = file[1:]
       for rows in self.rows:
           rows[-1] = int(rows[-1])


inventory = Inventory('laptops.csv') 
print(inventory.header)
len(inventory.rows)

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']


1303

## Updated Inventory Class to Look-up Laptop ID (1st Update)

In [3]:
# 2nd version of Inventory Class 

class Inventory():
    
   def __init__(self, csv_filename):
        
       with open(csv_filename) as f:
           reader = csv.reader(f)
           file = list(reader)
       self.header = file[0]
       self.rows = file[1:]
       for rows in self.rows:
           rows[-1] = int(rows[-1])

# Created a method to look up Laptop IDs             
            
   def get_laptop_from_id(self, laptop_id):    
       for rows in self.rows:
            if rows[0] == laptop_id:
                return rows
       return None
        
inventory = Inventory("laptops.csv")
print(inventory.get_laptop_from_id('3362737'))
print(inventory.get_laptop_from_id('3362736'))

# Business Example: Customer comes to store with receipt for laptop, we can quickly find the laptop and all details with ID

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


## 2nd Update for Inventory Class - ID Look-Up Method Time Complexity Improvement 

* Improved the time complexity by pre-processing the data into dictionary. 

In [4]:
# 3rd version of Inventory Class
# Added to the __init__() method, pre-processed the data into a dictionary, with the ID as a key
# Then created a get_laptop_from_id_fast() method, resulting in a faster search method

class Inventory():
    
   def __init__(self, csv_filename):
        
       with open(csv_filename) as f:
           reader = csv.reader(f)
           file = list(reader)
       self.header = file[0]
       self.rows = file[1:]
       for rows in self.rows:
           rows[-1] = int(rows[-1])
            
# Code below pre-processes the data into a dictionary (ID was the key)
            
       self.id_to_row = {}
       for rows in self.rows:
            self.id_to_row[rows[0]] = rows
            
   def get_laptop_from_id(self, laptop_id):
       for rows in self.rows:
            if rows[0] == laptop_id:
                return rows
       return None

# This resulted in a faster search method, time complexity went from O(R) to O(1) 

   def get_laptop_from_id_fast(self, laptop_id):
       if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
       return None

inventory = Inventory("laptops.csv")
print(inventory.get_laptop_from_id_fast('3362737'))
print(inventory.get_laptop_from_id_fast('3362736'))            

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


## Measuring performance of the two ID Look-Up Methods

In [5]:
# Measurement of the execution times of each Loop-Up Methods

import time
import random

ids = [str(random.randint(1000000, 9999999)) for _ in range(10000)]

inventory = Inventory("laptops.csv")

# time measurement with no dictionary

total_time_no_dict = 0 
for identifier in ids:
    start = time.time()
    inventory.get_laptop_from_id(identifier)
    end = time.time()
    total_time_no_dict += end - start

# time measurement with dictionary    
    
total_time_dict = 0
for identifier in ids:
    start = time.time()
    inventory.get_laptop_from_id_fast(identifier)
    end = time.time()
    total_time_dict += end - start
    
print(total_time_no_dict, total_time_dict)

0.7326846122741699 0.003718852996826172


Dividing the first method by the second shows that improving the time complexity of the method caused the performance of the method to run 226 times faster.

## 3rd Update Inventory Class - Laptop Promotion Look-Up

In [6]:
# 3rd version of Inventory Class 
# Added check_promotion_dollars() Method - checks if customer can buy 1 or 2 laptops with given dollar amount. 

class Inventory():
    
   def __init__(self, csv_filename):
        
       with open(csv_filename) as f:
           reader = csv.reader(f)
           file = list(reader)
       self.header = file[0]
       self.rows = file[1:]
       for rows in self.rows:
           rows[-1] = int(rows[-1])
       self.id_to_row = {}
       for rows in self.rows:
            self.id_to_row[rows[0]] = rows
            
   def get_laptop_from_id(self, laptop_id):
       for rows in self.rows:
            if rows[0] == laptop_id:
                return rows
       return None

  
   def get_laptop_from_id_fast(self, laptop_id):
       if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
       return None

# Method added below, Can customer buy up to two laptops, given dollar amount
   
   def check_promotion_dollars(self, dollars):
       for rows in self.rows:
            if rows[-1] == dollars:
                return True
       for rows1 in self.rows:
            for rows2 in self.rows:
                if rows1[-1] + rows2[-1] == dollars:
                    return True
       return False

inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars(1000))
print(inventory.check_promotion_dollars(442))            

True
False


## 4th Update Inventory Class - Improving Time Complexity for Laptop Promotion Look-Up

In [7]:
# 4th Version of Inventory Class 
# Added a faster version of check_promotion_dollars Method by pre-processing the data using the set() function in the __init__() method

class Inventory():
    
   def __init__(self, csv_filename):
        
       with open(csv_filename) as f:
           reader = csv.reader(f)
           file = list(reader)
       self.header = file[0]
       self.rows = file[1:]
       for rows in self.rows:
           rows[-1] = int(rows[-1])
       self.id_to_row = {}
       for rows in self.rows:
            self.id_to_row[rows[0]] = rows
       self.prices = set()
       for rows in self.rows:
            self.prices.add(rows[-1])
            
   def get_laptop_from_id(self, laptop_id):
       for rows in self.rows:
            if rows[0] == laptop_id:
                return rows
       return None

  
   def get_laptop_from_id_fast(self, laptop_id):
       if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
       return None

   
   def check_promotion_dollars(self, dollars):
       for rows in self.rows:
            if rows[-1] == dollars:
                return True
       for rows1 in self.rows:
            for rows2 in self.rows:
                if rows1[-1] + rows2[-1] == dollars:
                    return True
       return False


# Improved method for checking if customer can purchase laptop/s with dollar amount

   def check_promotion_dollars_fast(self, dollars):
       if dollars in self.prices:
            return True
       for price in self.prices:
            if dollars - price in self.prices:
                return True
       return False

inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars_fast(1000))
print(inventory.check_promotion_dollars_fast(442))

True
False


## Measuring Performance of two Laptop Promotion Look-Up Methods

In [8]:
# Similar way of measuring the previous methods was used. 

prices = [random.randint(100, 500) for _ in range(100)]

inventory = Inventory("laptops.csv")

total_time_no_set = 0
for p in prices:
    start = time.time()
    inventory.check_promotion_dollars(p)
    end = time.time()
    total_time_no_set += end - start
    
total_time_set = 0
for p in prices:
    start = time.time()
    inventory.check_promotion_dollars_fast(p)
    end = time.time()
    total_time_set += end - start
    
print(total_time_no_set, total_time_set)


15.278456449508667 0.004851102828979492


It is clear that pre-processing the data with the set() function improved the time complexity. 

## 5th Update Inventory Class - Find Laptops Within Customers Budget

1. add sorting of Inventory rows by laptop price when instantiating an Inventory class object
2. add binary search method
3. add another binary search method with improved time complexity

In [9]:
# 5th version of Inventory class
# help customer find all laptops given budget of D dollars
# in other words, write method to answer query: Given budget, find all laptops

# sort all laptops by price to then use binary search to find
# first laptop in the sorted list with a price larger than D

def row_price(row):
    return row[-1]

class Inventory():
    
   def __init__(self, csv_filename):
        
       with open(csv_filename) as f:
           reader = csv.reader(f)
           file = list(reader)
       self.header = file[0]
       self.rows = file[1:]
       for rows in self.rows:
           rows[-1] = int(rows[-1])
       self.id_to_row = {}
       for rows in self.rows:
            self.id_to_row[rows[0]] = rows
       self.prices = set()
       for rows in self.rows:
            self.prices.add(rows[-1])
       self.rows_by_price = sorted(self.rows, key=row_price)   # Added 
            
   def get_laptop_from_id(self, laptop_id):
       for rows in self.rows:
            if rows[0] == laptop_id:
                return rows
       return None

  
   def get_laptop_from_id_fast(self, laptop_id):
       if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
       return None

   
   def check_promotion_dollars(self, dollars):
       for rows in self.rows:
            if rows[-1] == dollars:
                return True
       for rows1 in self.rows:
            for rows2 in self.rows:
                if rows1[-1] + rows2[-1] == dollars:
                    return True
       return False

   
   def check_promotion_dollars_fast(self, dollars):
       if dollars in self.prices:
            return True
       for price in self.prices:
            if dollars - price in self.prices:
                return True
       return False

   
   def find_laptop_with_price(self, target_price):
       range_start = 0                                   
       range_end = len(self.rows_by_price) - 1                       
       while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            value = self.rows_by_price[range_middle][-1]
            if value == target_price:                            
                return range_middle                        
            elif value < target_price:                           
                range_start = range_middle + 1             
            else:                                          
                range_end = range_middle - 1 
       if self.rows_by_price[range_start][-1] != target_price:                  
            return -1                                      
       return range_start


# FINAL Binary Search Algorithm 

   def find_first_laptop_more_expensive(self, target_price):
       range_start = 0 
       range_end = len(self.rows_by_price) - 1                   
       while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            price = self.rows_by_price[range_middle][-1]
            if price > target_price:
                range_end = range_middle
            else:
                range_start = range_middle + 1
       if self.rows_by_price[range_start][-1] <= target_price:                  
            return -1                                   
       return range_start

inventory = Inventory("laptops.csv")

print(inventory.find_first_laptop_more_expensive(1000))
print(inventory.find_first_laptop_more_expensive(10000)) 

683
-1


## Conclusion

In this project, three functionalities were created:

1. Looking up laptops by their ID number
2. Seeing how many laptops a customer could afford given their budget
3. Determining the highest priced laptop at which the customer could not afford; determining all of the laptops within the customer's budget

The three built-in Python modules used were: csv, time, and random

In this project, data pre-processing was used to significantly improve the time efficiency of the methods.